In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bangkok-traffy/bangkok_traffy.csv
/kaggle/input/traffy/bangkok_traffy.csv


In [2]:
!pip install pythainlp --upgrade
!pip install -U hdbscan
!pip install python-crfsuite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 69.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 41.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.3 MB/s eta 0:00:0000:010:01


In [3]:
from sklearn.cluster import DBSCAN

In [4]:
df = pd.read_csv('/kaggle/input/bangkok-traffy/bangkok_traffy.csv' ,quotechar='"', on_bad_lines='skip')

/tmp/ipykernel_31/3284918269.py:1: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/kaggle/input/bangkok-traffy/bangkok_traffy.csv' ,quotechar='"', on_bad_lines='skip')


In [ ]:
df = df.dropna(subset=['comment'])

In [ ]:
df['comment'].isnull().sum()

In [ ]:
df = df.dropna(subset=['latitude', 'longitude'])

coords = df[['latitude', 'longitude']].to_numpy()
coords_rad = np.radians(coords)
epsilon = 0.015 / 6371  # ประมาณ 15 เมตร
db = DBSCAN(eps=epsilon, min_samples=3, algorithm='ball_tree', metric='haversine')
df['cluster'] = db.fit_predict(coords_rad)

In [ ]:
cluster_ids = (df['cluster'].max())
cluster_ids

In [ ]:
df_with_clustering = df[df['cluster'] > -1]

In [ ]:
df_with_clustering['cluster'].value_counts()

In [ ]:
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# สำหรับภาษาไทย
from pythainlp.tokenize import word_tokenize as thai_word_tokenize
from pythainlp.corpus import thai_stopwords

# โหลด NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

remove_list = [
    'นะคะ', 'คะ', 'ค่ะ', 'ครับ', 'คับ', 'นะครับ', 'ช่วย', 'ช่วยด้วย', 'แจ้ง', 
    'ขอ', 'กรุณา', 'ความ', 'ขอความกรุณา', 'รายละเอียด', 'สวัสดี', 'รบกวน', 'หน้า', 
    'เบื้องต้น',  'ทำ', 'ขอให้', 'ดำเนินการ', 'ตรวจสอบ', 'โดยรวม', 'อีกด้วย', 
    'ขนาดใหญ่', 'คอย', 'ขอบพระคุณ', 'ขอบคุณ', 'พิจารณา', 'ทำให้เกิด', 'ระยะเวลา', 
    'นาน', 'บ่อย', 'เริ่ม', 'ติดต่อ', 'หน่วยงาน', 'ผม', 
    'ฉัน', 'ดิฉัน', 'ตอนนี้', 'เรื่อง','ต้องการ','เป็นเวลา','เวลา','แบบ',
    'แบบนี้','นี้','ท่าน','คุณ','ต้องการ','ความช่วยเหลือ','ช่วยเหลือ','สอบถาม','ถาม','ความคืบหน้า',
    'ชัชชาติ','traffyfondue','team','ร้องเรียน','ทุกครั้งที่','ทุกครั้ง','นิด','นิดนึง','นิดหน่อย','เล็กน้อย',
    'นั้น','นี้','ตอนนี้','ตอนน้ัน'
    
]


def clean_sentence(sentence, remove_stopwords=False):
  
    # Convert to lowercase (เฉพาะภาษาอังกฤษ)
    sentence = sentence.lower()
    
    # Remove HTML tags
    sentence = re.sub(r'<.*?>', '', sentence)

    # Remove URLs
    sentence = re.sub(r'http\S+|www\S+|https\S+', '', sentence)

    # Remove email addresses
    sentence = re.sub(r'\S*@\S*', '', sentence)


    # Remove special characters and numbers (แต่เก็บสระไทยไว้)
    sentence = re.sub(r'[^a-zA-Zก-ฮะ-์\s]', '', sentence)

    # Remove extra whitespace
    sentence = re.sub(r'\s+', ' ', sentence).strip()


    word_tokens = thai_word_tokenize(sentence)

    
    # Remove stopwords
    if remove_stopwords:
        stop_words = set(thai_stopwords())


        word_tokens = [word for word in word_tokens if (word not in stop_words) and (word not in remove_list)]
    
    # รวมคำกลับเป็นประโยค
    sentence = ' '.join(word_tokens)
    
    return sentence


# Function to process a batch of sentences
def clean_sentences(sentences, remove_stopwords=False):
    """
    Clean a list of sentences
    
    Args:
        sentences (list): List of input sentences
        remove_stopwords (bool): Whether to remove stopwords
        lang (str): Language code ("en" for English, "th" for Thai)
        
    Returns:
        list: List of cleaned sentences
    """
    sentences = [str(s) for s in sentences]

    return [clean_sentence(sentence, remove_stopwords) for sentence in sentences]

In [ ]:
from sentence_transformers import SentenceTransformer
from pythainlp.tag import NER
from sklearn.feature_extraction.text import TfidfVectorizer
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
ner = NER("thainer")

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def get_representative_text(texts, embeddings):
    centroid = np.mean(embeddings, axis=0)
    sims = cosine_similarity([centroid], embeddings)[0]
    return texts[np.argmax(sims)]



In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
import hdbscan
import numpy as np
from tqdm import tqdm


cluster_list = []

clustering_model = AgglomerativeClustering(
        metric='precomputed',
        linkage='average',
        distance_threshold=0.5,  # หรือค่าที่คุณต้องการ
        n_clusters=None  # ต้องตั้งเป็น None ถ้าใช้ distance_threshold
    )

for target_cluster in tqdm(df_with_clustering['cluster'].unique()):


    target_df = df_with_clustering[df_with_clustering['cluster'] == target_cluster]
    target_comments = target_df['comment'].values.tolist()

    # ข้อความตัวอย่าง
    texts = target_comments  # หรือจะใส่ข้อความ list ก็ได้


    type_cleaned = target_df['type'].map(lambda e: " ".join([s.strip().strip("'") for s in str(e)[1:-1].split(',')] if pd.notnull(e) else [])).values.tolist()


    
    # --------- Step 1: Sentence Embedding ---------
    # model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = embedding_model.encode(texts, show_progress_bar=False)
    
    clean_texts = clean_sentences(texts,True)
    type_embedding = embedding_model.encode(type_cleaned, show_progress_bar=False)
    
    # --------- Step 2: Named Entity Recognition ---------
    
    def extract_named_entities(text):
        entities = ner.tag(text)
        named_entities = [word for word, tag in entities if tag != 'O']
        
        # หากไม่มี named entities ให้คืนค่าเป็นคำที่เป็นข้อความดั้งเดิม
        if not named_entities:
            return text  # หรืออาจจะคืนค่าที่เป็น string ว่างก็ได้
        return " ".join(named_entities)
    
    
    
    ner_features = [extract_named_entities(text) for text in clean_texts]


    # --------- Step 3: TF-IDF Vectorization of NER ---------
    if len(ner_features) == 0:
        print("NER features are empty after filtering!")
    else:
        # --------- Step 3: TF-IDF Vectorization of NER ---------
        vectorizer = TfidfVectorizer()
        try:
            ner_tfidf = vectorizer.fit_transform(ner_features).toarray()
        except ValueError:
            continue  # ข้ามกรณี vocabulary ว่าง
    
    
    # --------- Step 4: Combine Embedding + NER ---------
   
    combined_features = np.concatenate([embeddings,type_embedding,ner_tfidf], axis=1)
    
    
    
    
    # --------- Step 5: HDBSCAN Clustering ---------
    similarity_matrix = cosine_similarity(combined_features)
    
    similarity_matrix = similarity_matrix.astype(np.float64)
    distance_matrix = 1 - similarity_matrix

    

    
    
    labels = clustering_model.fit_predict(distance_matrix)  # ถ้าใช้ similarity ต้องแปลงเป็นระยะทาง
    df.loc[target_df.index, 'text_cluster'] = labels
    
    # --------- Step 6: Show results ---------
    from collections import defaultdict
    
    clustered_texts = defaultdict(list)
    
    for label, text in zip(labels, texts):
        clustered_texts[label].append(text)

    for sub_label in set(labels):
        # เลือก index ของข้อความในคลัสเตอร์ย่อย
        sub_indices = [i for i, l in enumerate(labels) if l == sub_label]
        
        sub_texts = [texts[i] for i in sub_indices]
        sub_embeddings = embeddings[sub_indices]  # เลือกจาก embedding เดิมที่ encode ไปแล้ว
        
        rep = get_representative_text(sub_texts, sub_embeddings)
        
        cluster_list.append({
            'cluster_id': target_cluster,
            'text_cluster': sub_label,
            'cluster_desc': rep
        })


In [ ]:
result = pd.DataFrame(cluster_list)

In [ ]:
result.to_csv("cluster_info_df.csv", index=False)
df.to_csv("df_with_clustering.csv", index=False)

In [ ]:
from sklearn.cluster import AgglomerativeClustering

model = AgglomerativeClustering(
    metric='precomputed',
    linkage='average',
    distance_threshold=0.5,  # หรือค่าที่คุณต้องการ
    n_clusters=None  # ต้องตั้งเป็น None ถ้าใช้ distance_threshold
)

labels = model.fit_predict(distance_matrix)  # ถ้าใช้ similarity ต้องแปลงเป็นระยะทาง

# --------- Step 6: Show results ---------
from collections import defaultdict

clustered_texts = defaultdict(list)

for label, text in zip(labels, texts):
    clustered_texts[label].append(text)

# แสดงผลเป็นกลุ่ม
for label, group in clustered_texts.items():
    print(f"\n[Cluster {label}]")
    # print(len(group))
    for t in group:
        print(f" - {t}")